<a href="https://colab.research.google.com/github/abhi-11nav/Flight-fare-prediction/blob/main/Flight_fare_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset Credits

https://www.kaggle.com/datasets/nikhilmittal/flight-fare-prediction-mh

In [539]:
# Importing libraries for data manipulation 

import pandas as pd
import numpy as np 


# Importing libraries for data visualization

import matplotlib.pyplot as plt 
%matplotlib inline

import numpy as np 

In [540]:
# Storing the path of the data in a variable

dataset_path = "/content/drive/MyDrive/Flight fare prediction/Data_Train.xlsx"

# TRAINING DATA PREPROCESSING


In [541]:
# Loading the dataset into a variable 

dataset = pd.read_excel(dataset_path)

In [542]:
dataset.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [543]:
# Checking for any missing vlaues in the data

dataset.isna().any()

Airline            False
Date_of_Journey    False
Source             False
Destination        False
Route               True
Dep_Time           False
Arrival_Time       False
Duration           False
Total_Stops         True
Additional_Info    False
Price              False
dtype: bool

We have missing values in "Route" and "Total_Stops" features

In [544]:
# Dealing with missing values

dataset[dataset["Total_Stops"].isna() == True]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


Since this is the only row with missing value. Let us drop it 

In [545]:
# Dropping the null value row 

dataset.drop(index=9039, inplace=True, axis=0)

In [546]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10682 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10682 non-null  object
 1   Date_of_Journey  10682 non-null  object
 2   Source           10682 non-null  object
 3   Destination      10682 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10682 non-null  object
 6   Arrival_Time     10682 non-null  object
 7   Duration         10682 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10682 non-null  object
 10  Price            10682 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 1001.4+ KB


## DEALING WITH CATEGORICAL VARIABLES

#### DATETIME FEATURES

In [547]:
# We are going to deal with features related to time in this section 

dataset["travel_day"] = pd.to_datetime(dataset["Date_of_Journey"], format="%d/%m/%Y").dt.day
dataset["travel_month"] = pd.to_datetime(dataset["Date_of_Journey"], format="%d/%m/%Y").dt.month

if len(pd.to_datetime(dataset["Date_of_Journey"], format="%d/%m/%Y").dt.year.value_counts()) > 1:
  dataset["travel_year"] = pd.to_datetime(dataset["Date_of_Journey"], format="%d/%m/%Y").dt.year  
else:
  print("Non-unique values found in year. Hence, no feature created\n")

dataset.drop(["Date_of_Journey"], axis=1, inplace = True)
print("Original feature dropped")

Non-unique values found in year. Hence, no feature created

Original feature dropped


In [548]:
dataset["Arrival_hour"] = pd.to_datetime(dataset["Arrival_Time"]).dt.hour
dataset["Arrival_minute"] = pd.to_datetime(dataset["Arrival_Time"]).dt.minute
print("Features for Arrival time created\n")

dataset["Departure_hour"] = pd.to_datetime(dataset["Dep_Time"]).dt.hour
dataset["Departure_minute"] = pd.to_datetime(dataset["Dep_Time"]).dt.minute 
print("Features for Departure time created\n")

dataset.drop(["Arrival_Time", "Dep_Time"], axis=1, inplace=True)
print("Original features dropped")

Features for Arrival time created

Features for Departure time created

Original features dropped


In [549]:
# Converting the feature into a list to better modify and extract the required features 

duration = list(dataset["Duration"])

In [550]:
string_lengths = []

for index in range(len(duration)):
  length = len(duration[index])

  if length not in string_lengths:
    string_lengths.append(length)
    string_lengths.append(duration[index])

print(string_lengths, "\n\n", int(len(string_lengths)/2),"different types of strings")

[6, '2h 50m', 3, '19h', 7, '15h 30m', 2, '3h', 5, '3h 5m'] 

 5 different types of strings


This shows that we have 5 different length strings and the way they are formatted. This gives us an idea on how to deal with this feature. Although these may not be the only types of string. There could be other formats but these are all the lengths we have

In [551]:
# Dealing with types of strings 

for index in range(len(duration)):
  if len(duration[index].split()) != 2:
    if "h" in duration[index]:
      duration[index] += " 0m"
    else:
      duration[index] = "0h "+duration[index]

Converted all the varibales in the string to similar format

In [552]:
duration_hours= []
duration_minutes = []

for index in range(len(duration)):

  duration_hours.append(int(duration[index].split(sep="h")[0]))
  duration_minutes.append(int(duration[index].split(sep='h')[1].split(sep="m")[0]))


dataset["duration_hours"] = duration_hours
dataset["duration_minutes"] = duration_minutes
print("Additional features created from Duration feature\n")

dataset.drop(["Duration"], axis=1, inplace=True)
print("Original feature dropped")

Additional features created from Duration feature

Original feature dropped


In [553]:
# Printing the head of the dataset

dataset.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,travel_day,travel_month,Arrival_hour,Arrival_minute,Departure_hour,Departure_minute,duration_hours,duration_minutes
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,24,3,1,10,22,20,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,1,5,13,15,5,50,7,25
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882,9,6,4,25,9,25,19,0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1 stop,No info,6218,12,5,23,30,18,5,5,25
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1 stop,No info,13302,1,3,21,35,16,50,4,45


#### STRING FEATURES

In [554]:
# Let us explore the Additional_info features

dataset["Additional_Info"].value_counts()

No info                         8344
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: Additional_Info, dtype: int64

As majority of the flights have no additional info, these are considerd to be null values. So, we are going to drop this feature

In [555]:
dataset.drop("Additional_Info", axis=1, inplace=True)

Additional_Info feature dropped

In [556]:
dataset["Airline"].value_counts()

Jet Airways                          3849
IndiGo                               2053
Air India                            1751
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: Airline, dtype: int64

We are going to drop the last four values as they are very less in number compared to other values 

In [557]:
# Gathering the indexes of the variables to be dropped

indexes = []

for features in [drop_features for drop_features in list(dataset["Airline"].value_counts()[-4:].index)]:
  for ind in dataset[dataset["Airline"] == features].index:
    indexes.append(ind)

In [558]:
# Dropping the variables with low count

dataset.drop(index=indexes, axis=0, inplace=True)

In [559]:
# Verification

dataset["Airline"].value_counts()

Jet Airways          3849
IndiGo               2053
Air India            1751
Multiple carriers    1196
SpiceJet              818
Vistara               479
Air Asia              319
GoAir                 194
Name: Airline, dtype: int64

In [560]:
# Peforming One hot encoding on Ariline feature and adding this to the table. Also, dropping the original feature

dataset = pd.concat([dataset, pd.get_dummies(dataset["Airline"], drop_first=True)], axis=1)
print("One hot encoding performed and additional features addded\n")

dataset.drop("Airline", axis=1, inplace=True)
print("Original feature dropped")

One hot encoding performed and additional features addded

Original feature dropped


In [561]:
dataset.head()

,Source,Destination,Route,Total_Stops,Price,travel_day,travel_month,Arrival_hour,Arrival_minute,Departure_hour,Departure_minute,duration_hours,duration_minutes,Air India,GoAir,IndiGo,Jet Airways,Multiple carriers,SpiceJet,Vistara
0,Banglore,New Delhi,BLR → DEL,non-stop,3897,24,3,1,10,22,20,2,50,0,0,1,0,0,0,0
1,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,7662,1,5,13,15,5,50,7,25,1,0,0,0,0,0,0
2,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,13882,9,6,4,25,9,25,19,0,0,0,0,1,0,0,0
3,Kolkata,Banglore,CCU → NAG → BLR,1 stop,6218,12,5,23,30,18,5,5,25,0,0,1,0,0,0,0
4,Banglore,New Delhi,BLR → NAG → DEL,1 stop,13302,1,3,21,35,16,50,4,45,0,0,1,0,0,0,0


In [562]:
dataset["Total_Stops"].value_counts()

1 stop      5607
non-stop    3488
2 stops     1518
3 stops       45
4 stops        1
Name: Total_Stops, dtype: int64

In [563]:
# Drop the feature with 4 stops

dataset.drop(index = dataset[dataset["Total_Stops"]=="4 stops"].index, axis=0, inplace=True)

In [564]:
stops = list(dataset["Total_Stops"])

for x in range(len(stops)):
  if stops[x] in ["1 stop", "non-stop"]:
    if stops[x].split(sep="stop")[0] == "non-":
      stops[x] = 0
    else:
      stops[x] = int(stops[x].split(sep="stop")[0])
  else:
    stops[x] = int(stops[x].split(sep="stops")[0])

dataset["Total_Stops"] = stops
print("Modifications Complete - Categorical features encoded")

Modifications Complete - Categorical features encoded


In [565]:
# Printing the head of the dataset

dataset.head()

,Source,Destination,Route,Total_Stops,Price,travel_day,travel_month,Arrival_hour,Arrival_minute,Departure_hour,Departure_minute,duration_hours,duration_minutes,Air India,GoAir,IndiGo,Jet Airways,Multiple carriers,SpiceJet,Vistara
0,Banglore,New Delhi,BLR → DEL,0,3897,24,3,1,10,22,20,2,50,0,0,1,0,0,0,0
1,Kolkata,Banglore,CCU → IXR → BBI → BLR,2,7662,1,5,13,15,5,50,7,25,1,0,0,0,0,0,0
2,Delhi,Cochin,DEL → LKO → BOM → COK,2,13882,9,6,4,25,9,25,19,0,0,0,0,1,0,0,0
3,Kolkata,Banglore,CCU → NAG → BLR,1,6218,12,5,23,30,18,5,5,25,0,0,1,0,0,0,0
4,Banglore,New Delhi,BLR → NAG → DEL,1,13302,1,3,21,35,16,50,4,45,0,0,1,0,0,0,0


In [566]:
dataset["Source"].value_counts()

Delhi       4521
Kolkata     2871
Banglore    2190
Mumbai       696
Chennai      380
Name: Source, dtype: int64

In [567]:
dataset["Destination"].value_counts()

Cochin       4521
Banglore     2871
Delhi        1264
New Delhi     926
Hyderabad     696
Kolkata       380
Name: Destination, dtype: int64

In [568]:
# Creating dummy variables for source and destination

source = pd.get_dummies(dataset[["Source"]], drop_first=True)

destination = pd.get_dummies(dataset[["Destination"]], drop_first=True)

In [569]:
# Concatenating the dummy vartiable list with dataset and dropping the original features

dataset = pd.concat([dataset, pd.concat([source, destination], axis=1)], axis=1)
print("Concatenation done")

dataset.drop(["Source", "Destination"], axis=1, inplace=True)
print("Original features dropped")

Concatenation done
Original features dropped


In [570]:
dataset.head()

,Route,Total_Stops,Price,travel_day,travel_month,Arrival_hour,Arrival_minute,Departure_hour,Departure_minute,duration_hours,...,Vistara,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,BLR → DEL,0,3897,24,3,1,10,22,20,2,...,0,0,0,0,0,0,0,0,0,1
1,CCU → IXR → BBI → BLR,2,7662,1,5,13,15,5,50,7,...,0,0,0,1,0,0,0,0,0,0
2,DEL → LKO → BOM → COK,2,13882,9,6,4,25,9,25,19,...,0,0,1,0,0,1,0,0,0,0
3,CCU → NAG → BLR,1,6218,12,5,23,30,18,5,5,...,0,0,0,1,0,0,0,0,0,0
4,BLR → NAG → DEL,1,13302,1,3,21,35,16,50,4,...,0,0,0,0,0,0,0,0,0,1


In [571]:
# We are going to drop the "Route" feature as "Total_stops" features resemebles similarity with this feature 

dataset.drop(['Route'], axis=1, inplace=True)

In [572]:
dataset.head()

,Total_Stops,Price,travel_day,travel_month,Arrival_hour,Arrival_minute,Departure_hour,Departure_minute,duration_hours,duration_minutes,...,Vistara,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,0,3897,24,3,1,10,22,20,2,50,...,0,0,0,0,0,0,0,0,0,1
1,2,7662,1,5,13,15,5,50,7,25,...,0,0,0,1,0,0,0,0,0,0
2,2,13882,9,6,4,25,9,25,19,0,...,0,0,1,0,0,1,0,0,0,0
3,1,6218,12,5,23,30,18,5,5,25,...,0,0,0,1,0,0,0,0,0,0
4,1,13302,1,3,21,35,16,50,4,45,...,0,0,0,0,0,0,0,0,0,1


# TEST DATA PREPRCOESSING  

In [573]:
# Path of test data

test_data_path = "/content/drive/MyDrive/Flight fare prediction/Test_set.xlsx"

In [574]:
# Loading the data into a varibale 

test_data = pd.read_excel(test_data_path)

In [575]:
test_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [576]:
# Exploring the value counts and performing necessary actins before data preprcoessing

test_data.drop(["Additional_Info", "Route"], axis=1, inplace=True)

In [577]:
test_data["Airline"].value_counts()

Jet Airways                          897
IndiGo                               511
Air India                            440
Multiple carriers                    347
SpiceJet                             208
Vistara                              129
Air Asia                              86
GoAir                                 46
Multiple carriers Premium economy      3
Vistara Premium economy                2
Jet Airways Business                   2
Name: Airline, dtype: int64

In [578]:
# First preprocessing step 

# Dropping some airline values in test_data

indexes = []

for features in [drop_features for drop_features in list(test_data["Airline"].value_counts()[-3:].index)]:
  for ind in test_data[test_data["Airline"] == features].index:
    indexes.append(ind)

test_data.drop(index=indexes, axis=0, inplace=True)

In [579]:
#Dropping a varibale in test_data

dataset.drop(index = test_data[test_data["Total_Stops"] == 4].index, axis=0, inplace=True)

In [580]:
dataset.columns

Index(['Total_Stops', 'Price', 'travel_day', 'travel_month', 'Arrival_hour',
       'Arrival_minute', 'Departure_hour', 'Departure_minute',
       'duration_hours', 'duration_minutes', 'Air India', 'GoAir', 'IndiGo',
       'Jet Airways', 'Multiple carriers', 'SpiceJet', 'Vistara',
       'Source_Chennai', 'Source_Delhi', 'Source_Kolkata', 'Source_Mumbai',
       'Destination_Cochin', 'Destination_Delhi', 'Destination_Hyderabad',
       'Destination_Kolkata', 'Destination_New Delhi'],
      dtype='object')

In [581]:
# Defining a function for preprocessing of test_data 

def preprocess_one(test_data):

  test_data["travel_day"] = pd.to_datetime(test_data["Date_of_Journey"], format="%d/%m/%Y").dt.day
  test_data["travel_month"] = pd.to_datetime(test_data["Date_of_Journey"], format="%d/%m/%Y").dt.month

  test_data["Arrival_hour"] = pd.to_datetime(test_data["Arrival_Time"]).dt.hour
  test_data["Arrival_minute"] = pd.to_datetime(test_data["Arrival_Time"]).dt.minute

  test_data["Departure_hour"] = pd.to_datetime(test_data["Dep_Time"]).dt.hour
  test_data["Departure_minute"] = pd.to_datetime(test_data["Dep_Time"]).dt.minute

  test_data.drop(["Date_of_Journey", "Arrival_Time", "Dep_Time"], axis=1, inplace=True)

  return test_data

In [582]:
test_data = preprocess_one(test_data)

In [583]:
# Second preprocessing step

duration = list(test_data["Duration"])

for index in range(len(duration)):
  if len(duration[index].split()) != 2:
    if "h" in duration[index]:
      duration[index] += " 0m"
    else:
      duration[index] = "0h "+ duration[index]

duration_hours= []
duration_minutes = []

for index in range(len(duration)):
  duration_hours.append(int(duration[index].split(sep="h")[0]))
  duration_minutes.append(int(duration[index].split(sep='h')[1].split(sep="m")[0]))

test_data["duration_hours"] = duration_hours
test_data["duration_minutes"] = duration_minutes

test_data.drop(["Duration"], axis=1, inplace=True)

In [584]:
# Final preprocessing step 

def preprocess_two(test_data):

  test_data = pd.concat([test_data, pd.get_dummies(test_data["Airline"], drop_first=True)], axis=1)
  test_data.drop(["Airline"], inplace=True, axis=1)


  stops = list(test_data["Total_Stops"])
  
  for index in range(len(stops)):
    if stops[index] in ["1 stop", "non-stop"]:
      if stops[index].split(sep="stop")[0] == "non-":
        stops[index] = 0
      else:
        stops[index] = int(stops[index].split(sep="stop")[0])
    else:
      stops[index] = int(stops[index].split(sep="stops")[0])

  test_data["Total_Stops"] = stops


  dummy = pd.concat([pd.get_dummies(test_data[['Source']], drop_first=True), pd.get_dummies(test_data[["Destination"]], drop_first=True)],axis=1)
  test_data = pd.concat([test_data, dummy], axis=1)

  test_data.drop(["Source","Destination"], inplace=True, axis=1)

  return test_data

In [585]:
test_data = preprocess_two(test_data)

In [586]:
test_data.columns

Index(['Total_Stops', 'travel_day', 'travel_month', 'Arrival_hour',
       'Arrival_minute', 'Departure_hour', 'Departure_minute',
       'duration_hours', 'duration_minutes', 'Air India', 'GoAir', 'IndiGo',
       'Jet Airways', 'Multiple carriers', 'SpiceJet', 'Vistara',
       'Source_Chennai', 'Source_Delhi', 'Source_Kolkata', 'Source_Mumbai',
       'Destination_Cochin', 'Destination_Delhi', 'Destination_Hyderabad',
       'Destination_Kolkata', 'Destination_New Delhi'],
      dtype='object')

In [587]:
dataset.columns

Index(['Total_Stops', 'Price', 'travel_day', 'travel_month', 'Arrival_hour',
       'Arrival_minute', 'Departure_hour', 'Departure_minute',
       'duration_hours', 'duration_minutes', 'Air India', 'GoAir', 'IndiGo',
       'Jet Airways', 'Multiple carriers', 'SpiceJet', 'Vistara',
       'Source_Chennai', 'Source_Delhi', 'Source_Kolkata', 'Source_Mumbai',
       'Destination_Cochin', 'Destination_Delhi', 'Destination_Hyderabad',
       'Destination_Kolkata', 'Destination_New Delhi'],
      dtype='object')

In [588]:
# Splitting the dataset into features and labels

X = dataset.drop(['Price'], axis = 1)
y = dataset["Price"]

In [589]:
from sklearn.preprocessing import MinMaxScaler

scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()

X = scaler1.fit_transform(X)

y = scaler2.fit_transform(np.array(y).reshape(-1,1))

In [590]:
# Train and validation splits 

from sklearn.model_selection import train_test_split 

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.18)

# MODEL TRAINING 

In [591]:
# Importing adaboost 

from sklearn.ensemble import AdaBoostRegressor 

# Importing libraries for hyper-parameter tuning 

from sklearn.model_selection import RandomizedSearchCV

In [592]:
model = AdaBoostRegressor(n_estimators= 8, learning_rate=0.5)

In [593]:
model.fit(train_X, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(learning_rate=0.5, n_estimators=8)

In [594]:
pred_val = model.predict(test_X)

In [595]:
from sklearn.metrics import mean_squared_error

In [596]:
mse = mean_squared_error(test_y, pred_val)

In [597]:
test_y

array([[0.14894379],
       [0.06030113],
       [0.16023141],
       ...,
       [0.23937664],
       [0.16023141],
       [0.12846025]])

In [598]:
pred_val

array([0.18482707, 0.17232277, 0.18482707, ..., 0.18482707, 0.18482707,
       0.17232277])

In [599]:
mse

0.002914382858328312

In [600]:
# Importing libraries for storing the model 

import pickle 

In [601]:
file = open("modek.pkl", "wb")

pickle.dump(model, file)

In [602]:
file_two = open("scaler.pkl", "wb")

pickle.dump(scaler2, file_two)